In [1]:
!python ../python/scripts/1_option_chain_build_historic_dataset_v1.0.py \
  --out-dir ./data/samples \
  --out-name options_chain_dataset.csv \
  --drops-name option_chain_historic_dataset_drops.csv \
  --tickers NVDA,AAPL,GOOGL,MSFT,META,AMZN,PLTR, \
  --start 2023-06-01 --end 2023-06-30 \
  --strike-range 0 \
  --min-band-strikes 6 \
  --min-band-prn-strikes 4 \
  --min-prn-train 0.08 --max-prn-train 0.92 \
  --prefer-bidask \
  --min-chain-used-hard 0 \
  --write-drops \
  --verbose-skips \
  --stock-source yfinance \
  --no-cache

[PLAN] Weeks=4 (Mondays) range=2023-06-01..2023-06-30 tickers=7 snapshots/day=4 (Mon-Thu)
[UNIVERSE] ['NVDA', 'AAPL', 'GOOGL', 'MSFT', 'META', 'AMZN', 'PLTR']
[CFG] pRN train band: [0.08, 0.92]
[CFG] band start=0.06 cap=0.1 step=0.01 adaptive=True
[CFG] min strikes: curve=6 after_pRN=4
[CFG] expiry fallback Fri->Sat: True
[CFG] split_adjustment: True
[CFG] prefer_bidask: True
[CFG] threads=6 cache=False stock_source=yfinance
[STOCK] Preloading closes for 2023-06-01..2023-07-04 ...
[STOCK] Preloading yfinance Close for 2023-05-25..2023-07-11 (tickers=7) ...
[STOCK] Fetching split events via yfinance ...
[PROGRESS] 100/112 jobs | groups_kept=80 | rows=673 | last=AAPL week=2023-06-26 asof_target=2023-06-28
[PROGRESS] 112/112 jobs | groups_kept=89 | rows=728 | last=PLTR week=2023-06-26 asof_target=2023-06-29
[SUMMARY] rows=728 tickers=6 weeks=5 groups=83
[SUMMARY] per-ticker groups:
ticker
AMZN     15
GOOGL    15
META     15
MSFT     15
PLTR     13
AAPL     10
[WRITE] ./data/samples/option

In [23]:
!python ./scripts/2_calibrate_logit_model_v1.2.py \
  --csv ./data/datasets/options_chain_dataset.csv \
  --out-dir ./data/models/v1_1_auto_weighted \
  --features x_logit_prn,T_days,abs_log_m,rv20 \
  --calibrate none \
  --fit-weight-renorm mean1

RUNNING SCRIPT: /Users/dorianc./Desktop/pm-edge/python/./scripts/2_calibrate_logit_model_v1.2.py

=== FEATURE PRESENCE ===
    feature  present   dtype  non_missing
x_logit_prn     True float64        36776
     T_days     True   int64        36776
  abs_log_m     True float64        36776
       rv20     True float64        36075

=== SPLIT NON-MISSING COUNTS (before dropping) ===
    feature  trainfit_nonmissing  calib_nonmissing  val_nonmissing  test_nonmissing
x_logit_prn                19380              5615            5965             5816
     T_days                19380              5615            5965             5816
  abs_log_m                19380              5615            5965             5816
       rv20                18679              5615            5965             5816

=== FINAL FEATURES USED FOR FIT ===
['x_logit_prn', 'T_days', 'abs_log_m', 'rv20']

=== FIT WEIGHT DEBUG ===
TRAIN_FIT: n=19380 sum_raw=3.032528 sum_fit=19380.000000
CALIB    : n=5615 sum_raw=0.

In [31]:
!python ./scripts/2_calibrate_logit_model_v1.5.py \
  --csv ./data/datasets/options_chain_dataset.csv \
  --out-dir ./data/models/v1_5_ticker_intercepts \
  --features x_logit_prn,T_days,abs_log_m,rv20 \
  --calibrate platt \
  --test-weeks 20 \
  --val-windows 4 \
  --val-window-weeks 10 \
  --fit-weight-renorm mean1 \
  --train-decay-half-life-weeks 52


RUNNING SCRIPT: /Users/dorianc./Desktop/pm-edge/python/./scripts/2_calibrate_logit_model_v1.5.py

=== TICKER SANITY ===
unique_tickers: 7

=== FEATURE PRESENCE (numeric) ===
    feature  present   dtype  non_missing
x_logit_prn     True float64        36776
     T_days     True   int64        36776
  abs_log_m     True float64        36776
       rv20     True float64        36075

=== ROLLING WINDOW SPECS ===
roll_val_1: window=[2024-11-29..2025-01-31] (10 wks), train uses weeks < 2024-11-29
roll_val_2: window=[2025-02-07..2025-04-11] (10 wks), train uses weeks < 2025-02-07
roll_val_3: window=[2025-04-25..2025-06-27] (10 wks), train uses weeks < 2025-04-25
roll_val_4: window=[2025-07-11..2025-09-12] (10 wks), train uses weeks < 2025-07-11
FINAL TEST block: [2025-09-19..2026-01-30] (20 wks)

=== ROLLING SELECTION SUMMARY (best by avg model logloss) ===
     C  avg_roll_logloss_model  avg_roll_logloss_baseline  avg_roll_delta  n_windows_used
 0.001                0.533421               